## Víznerov kvantni novac

Stiven Vízner je 1968. godine osmislio koncept **kvantnog novca**, ideju koja do danas nije realizovana zbog nepostojanja kvantne memorije. Osnovna ideja je da se kvantni novčić može predstaviti kao kubit pripremljen u jednoj od dve moguće baze (standardna ili dijagonalna). Svaki kvantni novčić izdaje se zajedno sa jedinstvenim **serijskim brojem**, koji se beleži u banci. Ovaj broj služi kao identifikator koji povezuje kvantni novčić koji poseduje korisnik sa odgovarajućim zapisom o pripremi koji je sačuvan u bazi banke. Pošto se nepoznata kvantna stanja ne mogu klonirati (teorema o zabrani kloniranja), falsifikovani novčići ne mogu se pouzdano reprodukovati.
U Víznerovom originalnom predlogu, banka beleži samo tajnu bazu pripreme za svaki kubit, koja se kasnije koristi za proveru autentičnosti.  

U ovoj simulaciji, i oznaka pripreme i stvarno kvantno stanje čuvaju se u bazi podataka banke. To omogućava verifikaciju direktnim poređenjem vernosti (fidelity) primljenog novčića sa referentnim stanjem banke.  

![QuantumMoney](images/QuantumMoney.png)

Ova demonstracija dokaza koncepta prikazuje:
- **Kreiranje novčića:** Banka nasumično bira jedno od četiri stanja ($\lvert 0\rangle, \lvert 1\rangle, \lvert +\rangle, \lvert -\rangle$) i čuva ga u svojoj evidenciji.  
- **Novčanici:** Ana i Boban imaju svoje „novčanike“ u kojima čuvaju kvantne novčiće.  
- **Izdavanje:** Ana može da zatraži novčiće od banke.  
- **Verifikacija:** Novčići se proveravaju u odnosu na referentno stanje banke pomoću vernosti.  
- **Prenos:** Novčići se mogu prenositi između Ane i Bobana ako prođu proveru autentičnosti.  

*Literatura:*  
S. Wiesner, *Conjugate Coding*, SIGACT News, vol. 15, br. 1, str. 78–88, 1983. (originalno napisano 1968.)

---

### Zadatak
1. Pokrenite program i posmatrajte kako se kubit, koji predstavlja kvantni novčić, izdaje, čuva u novčaniku i prenosi.  
2. Proučite funkcije i objasnite kako:  
   - `prepare_coin_state` generiše kvantno stanje,  
   - `create_quantum_coin_pair` kreira kopiju za banku i korisnika,  
   - `verify_coin` proverava autentičnost pomoću vernosti.  
3. Objasnite zašto je vernost neophodna pri verifikaciji i zašto kubit ne može biti kloniran klasično.  

### Očekivani rezultat
- Kada se program pokrene, banka izdaje dva novčića Ani i beleži njihove serijske brojeve.  
- Anin novčanik u početku sadrži oba novčića, dok je Bobanov prazan.  
- Nakon uspešnog prenosa, jedan novčić se uspešno premešta iz Aninog u Bobanov novčanik, a vrednost vernosti potvrđuje njegovu autentičnost.  
- Konačno stanje je da Ana zadržava jedan novčić, dok Boban poseduje preneti novčić.  
- Ako se pre verifikacije uvede greška ili izvrši manipulacija, vernost opada ispod granične vrednosti.  U tom slučaju, pokušaj prenosa neuspeva, novčić se odbacuje i ostaje evidentiran u Aninom novčaniku.  

### Eksperimenti
Moguća proširenja programa uključuju:  
- Izdavanje dodatnih novčića i posmatranje kako se generišu jedinstveni serijski brojevi.  
- Namerno oštećivanje kvantnih novčića primenom nasumičnih operacija (npr. `x`, `h`) pre verifikacije i posmatranje promene vernosti.  
- Podešavanje praga vernosti (trenutno `0.99`) kako bi se ispitalo kako to utiče na prihvatanje ili odbijanje.  
- Omogućavanje Bobanu da direktno zatraži novčiće od banke.  
- Proširenje sistema na **višekubitne novčiće** tako što će se funkcija `prepare_coin_state` prilagoditi za rad sa više kubita.  
- Evidentiranje svih transakcija u dnevniku radi simulacije jednostavnog blockchain sistema za kvantni novac.  

---

**Pitanje za razmišljanje:**  
Zašto teorema o zabrani kloniranja onemogućava falsifikatoru da napravi kopiju kvantnog novčića sa savršenom vernošću?  


In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, state_fidelity
import random

# -- Bank ledger and user wallets --
bank_ledger  = {}
Alice_wallet = {}
Bob_wallet   = {}

# -- Quantum state preparation --
def prepare_coin_state(prep):
    """
    Given a string label ('0', '1', '+', '-'), 
    return the corresponding single-qubit Statevector.
    """
    qc = QuantumCircuit(1)
    if prep == '0':
        pass
    elif prep == '1':
        qc.x(0)
    elif prep == '+':
        qc.h(0)
    elif prep == '-':
        qc.x(0)
        qc.h(0)
    state = Statevector(qc)
    return state

# -- Quantum coin creation --
def create_quantum_coin_pair():
    """
    One copy of the state is kept by the bank; 
    one copy is issued to the user.
    """
    # Step 1: Randomly choose one of the four states.
    states = ['0', '1', '+', '-']
    chosen_state = random.choice(states)
    
    # Step 2: Prepare the quantum coin.
    state = prepare_coin_state(chosen_state)
    
    # Step 3: Generate a unique serial number and store it in the bank ledger.
    while (serial := random.randint(1000, 9999)) in bank_ledger:
        continue

    bank_ledger[serial] = {
        'state': state,
        'preparation': chosen_state
    }

    # Step 4: Regenerate the state with the same preparation (user copy).
    state = prepare_coin_state(chosen_state)
    return serial, state

# -- Request a coin from the bank --
def request_coin(user_ledger):
    serial, state = create_quantum_coin_pair()
    user_ledger[serial] = state
    return serial

# Get the list of coin serial numbers in a user's wallet.
def list_user_coins(wallet):
    return list(wallet.keys())
    
# -- Coin verification --
def verify_coin(serial, user_state):
    """
    Compare the user's state to the bank's reference 
    using state fidelity (threshold 0.99).
    """
    if serial not in bank_ledger:
        return False, "Invalid serial number"
    
    bank_state = bank_ledger[serial]['state']
    fidelity = state_fidelity(bank_state, user_state)
    return fidelity >= 0.99, fidelity

# -- Transfer a coin between users --
def transfer_coin(serial, from_ledger, to_ledger):
    if serial not in from_ledger:
        print("Transfer failed: coin not found in sender's ledger.")
        return False
    
    user_coin = from_ledger[serial]
    valid, fidelity = verify_coin(serial, user_coin)
    
    if valid:
        # Move coin.
        to_ledger[serial] = user_coin
        del from_ledger[serial]
        print(f"Transfer successful. Fidelity: {fidelity:.4f}")
        return True
    else:
        print(f"Transfer failed: Fidelity: {fidelity:.4f}")
        return False

# ------------------------------------------------
#                main program
# ------------------------------------------------

# Alice requests two coins.
serial_1 = request_coin(Alice_wallet)
print(f"Coin {serial_1} issued.")
serial_2 = request_coin(Alice_wallet)
print(f"Coin {serial_2} issued.")

# Alice and Bob list their coins.
coins = list_user_coins(Alice_wallet)
print(f"Alice's coins: {coins}")

coins = list_user_coins(Bob_wallet)
print(f"Bob's coins: {coins}")

# Alice transfers a coin to Bob.
transfer_status = transfer_coin(serial_2, Alice_wallet, Bob_wallet)

# Alice and Bob list their coins again.
coins = list_user_coins(Alice_wallet)
print(f"Alice's coins: {coins}")

coins = list_user_coins(Bob_wallet)
print(f"Bob's coins: {coins}")
